In [1]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
def read_df(filename):
    
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_df("yellow_tripdata_2022-01.parquet")
df_valid = read_df("yellow_tripdata_2022-02.parquet")

In [8]:
len(df_train), len(df_valid)

(2421440, 2918187)

In [9]:
df_train["PUDO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_valid["PUDO"] = df_valid["PULocationID"] + "_" + df_valid["DOLocationID"]

In [10]:
categorical = ["PUDO"]
numerical = ["trip_distance"]

In [11]:
dv = DictVectorizer()

train_data = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_data)

In [12]:
valid_data = df_valid[categorical + numerical].to_dict(orient="records")
X_valid = dv.transform(valid_data)

In [13]:
y_train = df_train["duration"].values
y_valid = df_valid["duration"].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)

mse = mean_squared_error(y_valid, y_pred, squared=False)
mse

5.530252669479954

In [ ]:
with open("models/linear_regression_v2.bin", "wb") as fout:
    pickle.dump((dv, lr), fout)

In [20]:
lr = Lasso(alpha=0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)

mse = mean_squared_error(y_valid, y_pred, squared=False)
mse

9.374921127169834

In [21]:
with open("models/lasso_v1.bin", "wb") as fout:
    pickle.dump((dv, lr), fout)

In [22]:
lr = Ridge(alpha=0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)

mse = mean_squared_error(y_valid, y_pred, squared=False)
mse

6.00738228185992

In [23]:
with open("models/ridge_v1.bin", "wb") as fout:
    pickle.dump((dv, lr), fout)